<a href="https://colab.research.google.com/github/kevinmoreno1-ux/ENTREGAS/blob/main/02-preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

# Especificar la ruta al archivo CSV
file_path = '/content/drive/MyDrive/IA/SaberPro/train.csv'

# Leer el archivo CSV
print("=== PASO 1: CARGA INICIAL DEL DATASET ===")
data = pd.read_csv(file_path)
print(f"✓ Dataset cargado: {data.shape[0]} filas × {data.shape[1]} columnas")
print("Primeras 5 filas:")
print(data.head())
print("\n" + "="*50)

# 2. ANÁLISIS INICIAL DE DATOS FALTANTES
print("\n=== PASO 2: ANÁLISIS DE VALORES FALTANTES INICIALES ===")
missing_initial = data.isnull().sum()
missing_percent_initial = (missing_initial / len(data)) * 100

missing_info_initial = pd.DataFrame({
    'Valores_Faltantes_Iniciales': missing_initial,
    'Porcentaje_Faltante_Inicial': missing_percent_initial
})

print("Resumen de valores faltantes por columna:")
for col in data.columns:
    if missing_initial[col] > 0:
        print(f"  - {col}: {missing_initial[col]} faltantes ({missing_percent_initial[col]:.2f}%)")
    else:
        print(f"  - {col}: Sin valores faltantes")

print(f"\nTotal de valores faltantes iniciales: {missing_initial.sum()}")
print("\n" + "="*50)

# 3. CONVERSIÓN DE COLUMNAS CATEGÓRICAS A ONE-HOT ENCODING
print("\n=== PASO 3: ONE-HOT ENCODING DE VARIABLES CATEGÓRICAS ===")

# Identificar columnas categóricas para one-hot encoding
columns_to_encode = [
    'E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO', 'E_VALORMATRICULAUNIVERSIDAD',
    'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET',
    'F_EDUCACIONPADRE', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL',
    'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1',
    'F_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL'
]

print("Columnas seleccionadas para one-hot encoding:")
for i, col in enumerate(columns_to_encode, 1):
    if col in data.columns:
        unique_vals = data[col].nunique()
        print(f"  {i}. {col} (valores únicos: {unique_vals})")

# Aplicar one-hot encoding paso a paso
data_encoded = data.copy()
print("\nAplicando one-hot encoding...")

for col in columns_to_encode:
    if col in data_encoded.columns:
        before_shape = data_encoded.shape
        data_encoded = pd.get_dummies(data_encoded, columns=[col], prefix=[col], dummy_na=True)
        after_shape = data_encoded.shape
        new_cols = after_shape[1] - before_shape[1]
        print(f"  ✓ {col}: agregó {new_cols} nuevas columnas")

print(f"\nDimensiones después de one-hot encoding: {data_encoded.shape}")
print(f"Columnas totales: {len(data_encoded.columns)}")
print("\nPrimeras 3 columnas nuevas como ejemplo:")
print(data_encoded.columns[:3].tolist())
print("\n" + "="*50)

# 4. NORMALIZACIÓN DE VALORES NUMÉRICOS
print("\n=== PASO 4: NORMALIZACIÓN DE VARIABLES NUMÉRICAS ===")

# Identificar columnas numéricas para normalizar
numeric_columns_to_normalize = ['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']

print("Columnas numéricas a normalizar:")
for col in numeric_columns_to_normalize:
    if col in data_encoded.columns:
        stats = data_encoded[col].describe()
        print(f"  - {col}: media={stats['mean']:.3f}, std={stats['std']:.3f}, min={stats['min']:.3f}, max={stats['max']:.3f}")

# Aplicar normalización
scaler = StandardScaler()
print("\nAplicando StandardScaler (media=0, std=1)...")

for col in numeric_columns_to_normalize:
    if col in data_encoded.columns:
        original_values = data_encoded[col].copy()
        data_encoded[col] = scaler.fit_transform(data_encoded[[col]])

        # Mostrar comparación
        new_stats = data_encoded[col].describe()
        print(f"  ✓ {col}:")
        print(f"     Antes - media: {original_values.mean():.3f}, std: {original_values.std():.3f}")
        print(f"     Después - media: {new_stats['mean']:.3f}, std: {new_stats['std']:.3f}")

print("\n" + "="*50)

# 5. IMPUTACIÓN DE DATOS FALTANTES
print("\n=== PASO 5: IMPUTACIÓN DE DATOS FALTANTES ===")

# Verificar valores faltantes después de las transformaciones
missing_after_transform = data_encoded.isnull().sum()
missing_cols_after = missing_after_transform[missing_after_transform > 0]

if len(missing_cols_after) > 0:
    print("Columnas con valores faltantes antes de la imputación:")
    for col, count in missing_cols_after.items():
        print(f"  - {col}: {count} faltantes ({(count/len(data_encoded))*100:.2f}%)")

    # Separar columnas numéricas y categóricas para imputación
    numeric_cols = data_encoded.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = data_encoded.select_dtypes(include=['object']).columns.tolist()

    print(f"\nEstrategias de imputación:")
    print(f"  - Columnas numéricas ({len(numeric_cols)}): Mediana")
    print(f"  - Columnas categóricas ({len(categorical_cols)}): Moda (valor más frecuente)")

    # Imputar numéricos
    if numeric_cols:
        numeric_imputer = SimpleImputer(strategy='median')
        data_encoded[numeric_cols] = numeric_imputer.fit_transform(data_encoded[numeric_cols])
        print("  ✓ Imputación numérica completada")

    # Imputar categóricos
    if categorical_cols:
        categorical_imputer = SimpleImputer(strategy='most_frequent')
        data_encoded[categorical_cols] = categorical_imputer.fit_transform(data_encoded[categorical_cols])
        print("  ✓ Imputación categórica completada")

    # Verificar después de imputación
    missing_final = data_encoded.isnull().sum().sum()
    print(f"\nValores faltantes después de imputación: {missing_final}")
else:
    print("✓ No hay valores faltantes para imputar")

print("\n" + "="*50)

# 6. LABEL ENCODING PARA VARIABLES CATEGÓRICAS RESTANTES
print("\n=== PASO 6: LABEL ENCODING PARA VARIABLES CATEGÓRICAS RESTANTES ===")

remaining_categorical_cols = data_encoded.select_dtypes(include=['object']).columns.tolist()

if remaining_categorical_cols:
    print("Columnas categóricas restantes para label encoding:")
    label_encoders = {}

    for col in remaining_categorical_cols:
        unique_vals = data_encoded[col].nunique()
        sample_vals = data_encoded[col].unique()[:3]  # Mostrar primeros 3 valores
        print(f"  - {col}: {unique_vals} valores únicos, ej: {sample_vals}")

        # Aplicar label encoding
        le = LabelEncoder()
        data_encoded[col] = le.fit_transform(data_encoded[col].astype(str))
        label_encoders[col] = le
        print(f"    ✓ Codificada → valores: {len(le.classes_)}")

    print(f"\nTotal de columnas con label encoding: {len(label_encoders)}")
else:
    print("✓ No hay columnas categóricas restantes para codificar")

print("\n" + "="*50)

# 7. VERIFICACIÓN FINAL Y RESUMEN
print("\n=== PASO 7: VERIFICACIÓN FINAL Y RESUMEN ===")

print("INFORMACIÓN DEL DATASET PREPROCESADO:")
print(f"✓ Dimensiones finales: {data_encoded.shape}")
print(f"✓ Tipo de datos finales:")
print(data_encoded.dtypes.value_counts())

print(f"\n✓ Valores faltantes finales: {data_encoded.isnull().sum().sum()}")

# Estadísticas de las columnas normalizadas
print(f"\nESTADÍSTICAS DE COLUMNAS NORMALIZADAS:")
for col in numeric_columns_to_normalize:
    if col in data_encoded.columns:
        stats = data_encoded[col].describe()
        print(f"  {col}: media={stats['mean']:.3f}, std={stats['std']:.3f}")

# Resumen de transformaciones
print(f"\nRESUMEN DE TRANSFORMACIONES APLICADAS:")
print(f"  • Dataset original: {data.shape[0]} filas × {data.shape[1]} columnas")
print(f"  • Dataset preprocesado: {data_encoded.shape[0]} filas × {data_encoded.shape[1]} columnas")
print(f"  • Columnas one-hot encoding: {len(columns_to_encode)}")
print(f"  • Columnas normalizadas: {len(numeric_columns_to_normalize)}")
print(f"  • Columnas label encoding: {len(remaining_categorical_cols)}")
print(f"  • Valores faltantes iniciales: {missing_initial.sum()}")
print(f"  • Valores faltantes finales: {data_encoded.isnull().sum().sum()}")

print("\n" + "="*50)

# 8. MUESTRA DE DATOS FINALES
print("\n=== PASO 8: MUESTRA DE DATOS PREPROCESADOS ===")
print("Primeras 3 filas con algunas columnas transformadas:")

# Seleccionar algunas columnas para mostrar
sample_cols = []
for col in data_encoded.columns:
    if any(ind in col for ind in ['INDICADOR', 'RENDIMIENTO', 'ESTRATO']):
        sample_cols.append(col)
    if len(sample_cols) >= 8:  # Mostrar máximo 8 columnas
        break

if sample_cols:
    display(data_encoded[sample_cols].head(3))
else:
    print(data_encoded.iloc[:, :8].head(3))

print("\n¡PREPROCESADO COMPLETADO EXITOSAMENTE! ✅")

Mounted at /content/drive
=== PASO 1: CARGA INICIAL DEL DATASET ===
✓ Dataset cargado: 692500 filas × 21 columnas
Primeras 5 filas:
       ID  PERIODO_ACADEMICO            E_PRGM_ACADEMICO E_PRGM_DEPARTAMENTO  \
0  904256              20212                  ENFERMERIA              BOGOTÁ   
1  645256              20212                     DERECHO           ATLANTICO   
2  308367              20203       MERCADEO Y PUBLICIDAD              BOGOTÁ   
3  470353              20195  ADMINISTRACION DE EMPRESAS           SANTANDER   
4  989032              20212                  PSICOLOGIA           ANTIOQUIA   

                E_VALORMATRICULAUNIVERSIDAD E_HORASSEMANATRABAJA  \
0  Entre 5.5 millones y menos de 7 millones    Menos de 10 horas   
1  Entre 2.5 millones y menos de 4 millones                    0   
2  Entre 2.5 millones y menos de 4 millones      Más de 30 horas   
3  Entre 4 millones y menos de 5.5 millones                    0   
4  Entre 2.5 millones y menos de 4 millones  En

,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,F_ESTRATOVIVIENDA_Estrato 1,F_ESTRATOVIVIENDA_Estrato 2,F_ESTRATOVIVIENDA_Estrato 3,F_ESTRATOVIVIENDA_Estrato 4
0,0.437002,-0.556223,0.813978,0.060296,False,False,True,False
1,0.346934,-0.481341,0.508180,0.016142,False,False,True,False
2,0.232301,-0.492038,0.729034,0.016142,False,False,True,False



¡PREPROCESADO COMPLETADO EXITOSAMENTE! ✅
